In [ ]:
from dataretrieval import nwis
import geopandas as gpd
from pathlib import Path
import numpy as np
import pandas as pd
import json
import os

In [ ]:
proj_dir = Path("../../../")

data_dir = proj_dir / "Data/insitu/conditions"
data_dir.mkdir(exist_ok=True)

# load metadata
stations_metadata_path = Path(proj_dir, "Data/insitu/metadata/stations.csv")
stations_attributes_path = Path(proj_dir, "Data/insitu/metadata/dictionaries/stations_attributes.csv")

stations_attributes = pd.read_csv(stations_attributes_path)

if not os.path.exists(stations_metadata_path):
    stations_metadata = pd.DataFrame(columns=stations_attributes['Attribute_name'])
    stations_metadata.to_csv(stations_metadata_path, index=False)

stations_metadata = pd.read_csv(stations_metadata_path)

target_parameters = {
    "00010_Maximum": "max_temp(C)",
    "00010_Minimum": "min_temp(C)",
    "00010_Mean": "avg_temp(C)",
    "00060_Mean": "avg discharge (cfs)",
}

parameter_codes = {
    "max water temperature (C)": "Maximum water temperature, degrees Celsius",
    "min water temperature (C)": "Minimum water temperature, degrees Celsius",
    "avg water temperature (C)": "Mean water temperature, degrees Celsius",
    "avg discharge (cfs)": "Discharge, cubic feet per second",
    "avg discharge (m3/d)": "Discharge, cubic meters per day",
}

startDt = "1982-08-01"
endDt = "2024-02-26"

In [ ]:
# Load the basin vector data
basin_gdf = gpd.read_file(proj_dir / "data/gis/geopackages/columbia_river_basin.gpkg", layer="Basins")

# divide the bounding box into 16 smaller boxes
xmin, ymin, xmax, ymax = basin_gdf.bounds.values[0]
x = np.linspace(xmin, xmax, 5)
y = np.linspace(ymin, ymax, 5)

# create a list of lists for the bounding boxes
bb = []
for i in range(len(x) - 1):
    for j in range(len(y) - 1):
        bb.append(list(np.array([x[i], y[j], x[i + 1], y[j + 1]]).round(6)))

In [ ]:
# get the sites for each of the bounding boxes and save them to a dataframe
siteList = []
for i in range(len(bb)):
    try:
        siteList.append(nwis.what_sites(bBox=bb[i], startDt=startDt, endDt=endDt, parameterCd='00010')[0])
    except:
        pass

In [ ]:
siteList_df = pd.concat(siteList, ignore_index=True)

filtered_sites = gpd.GeoDataFrame(siteList_df, geometry=gpd.points_from_xy(siteList_df['dec_long_va'], siteList_df['dec_lat_va']), crs='epsg:4326')
sites_within_basin = filtered_sites[filtered_sites.within(basin_gdf.geometry[0])]
# sites_within_basin.head()

In [ ]:
# # save sites_within_basin to a csv file
# sites_within_basin.to_csv(Path(data_dir, "processed", "sites_within_basin2.csv"), index=False)

# Download the data of filtered sites


In [ ]:
# for site in sites_within_basin["site_no"]:
for site in ['12301933']:
    # for site in ["14070615", "14070620", "14070621"]:
    try:
        site_data = nwis.get_record(sites=site, service="dv", start=startDt, end=endDt)
        site_data.index.rename("date", inplace=True)
        column_dict = {
            col: target_parameters[col]
            for col in site_data.columns
            if col in target_parameters.keys()
        }
        site_data.rename(
            columns=column_dict,
            inplace=True,
        )
        if "avg discharge (cfs)" in site_data.columns:
            site_data["outflow(m3/d)"] = (
                site_data["avg discharge (cfs)"] * 0.0283168 * 86400
            )
            column_dict["outflow(m3/d)"] = "outflow(m3/d)"

        if column_dict:
            site_data[column_dict.values()].to_csv(
                Path(data_dir, "processed", f"USGS_{site}.csv"), index=True
            )

            # update metadata
            station_ID = "USGS_" + site.upper()

            if station_ID not in stations_metadata["station_ID"].values:
                stations_metadata = pd.concat(
                    [
                        stations_metadata,
                        pd.DataFrame(
                            {
                                "station_ID": [station_ID],
                                "id_at_source": [site.upper()],
                                "available_data": ["{}"],
                                "source_URL": ['{"url" : []}'],
                                "description": [
                                    sites_within_basin[
                                        sites_within_basin["site_no"] == site
                                    ]["station_nm"].values[0]
                                ],
                                "latitude": [
                                    sites_within_basin[
                                        sites_within_basin["site_no"] == site
                                    ]["dec_lat_va"].values[0]
                                ],
                                "longitude": [
                                    sites_within_basin[
                                        sites_within_basin["site_no"] == site
                                    ]["dec_long_va"].values[0]
                                ],
                                "site_params": ["{}"],
                            }
                        ),
                    ],
                    ignore_index=True,
                )

            # update source url
            source_url = json.loads(
                stations_metadata.loc[
                    stations_metadata["station_ID"] == station_ID, "source_URL"
                ].values[0]
            )

            if (
                f"https://waterdata.usgs.gov/monitoring-location/{site}"
                not in source_url["url"]
            ):
                source_url["url"].append(
                    f"https://waterdata.usgs.gov/monitoring-location/{site}"
                )
                stations_metadata.loc[
                    stations_metadata["station_ID"] == station_ID, "source_URL"
                ] = json.dumps(source_url)

            # update the available data
            availble_data = stations_metadata.loc[
                stations_metadata["station_ID"] == station_ID, "available_data"
            ].values[0]
            availble_data = json.loads(availble_data)

            # check if there is "conditions"  in the available data
            if "conditions" not in availble_data.values():
                availble_data["conditions"] = []
            # add the parameters to the available data
            # print(parameters[1:])
            for param in column_dict.values():
                if param not in availble_data["conditions"]:
                    availble_data["conditions"].append(param)

            # update the metadata
            stations_metadata.loc[
                stations_metadata["station_ID"] == station_ID, "available_data"
            ] = json.dumps(availble_data)

            # save the metadata
            stations_metadata.to_csv(stations_metadata_path, index=False)
    except:
        raise

In [ ]:
# add last updated date and last updated by
metadata_status = {
    "last_updated": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"),
    "update_message": "Updated the metadata to for USGS stations",
    "last_updated_by": "George Darkwah",
    "last_updated_by_email": "gdarkwah@uw.edu",
}

# save metadata
with open(Path(proj_dir, "Data/insitu/metadata/metadata_status.csv"), "w") as f:
    json.dump(metadata_status, f, indent=4)